In [1]:
import numpy as np

In [9]:
arr = np.array([[1, 2],
                [3, 4],
                [5, 6]])

s = arr.sum(axis=1)
s

array([ 3,  7, 11])

In [10]:
mask = np.logical_and(s >= 5, s <= 10)
mask

array([False,  True, False])

In [12]:
arr.shape

(3, 2)

In [13]:
arr[mask, :]

array([[3, 4]])